In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

## Filter with the Manhattan boundary

Please do not start this part if you have the `building_manhattan.geojson` file

In [2]:
# import dataset (from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
# currently we sampled entire dataset due to submit
gdf_building = gpd.read_file('../../data/raw/building/building_sample.geojson', dtype={'mpluto_bbl':str})

In [4]:
# filtering buildings with the Manhattan boundary
gdf_manhattan = gpd.read_file('../../data/raw/boundary/Borough Boundaries.geojson')
gdf_manhattan = gdf_manhattan.loc[gdf_manhattan.loc[:,'boro_name'] == 'Manhattan']

gdf_building_manhattan = gpd.sjoin(gdf_building, gdf_manhattan, how='inner', op='intersects')
gdf_building_manhattan = gdf_building_manhattan.drop(['index_right', 'boro_code', 'boro_name', 'shape_leng'], axis=1)

c:\Users\seung\anaconda3\envs\teamgigo\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


## Filter by Land use

In [12]:
# import dataset (from https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page)
# currently we sampled entire dataset due to submit
gdf_pluto_manhattan = gpd.read_file('../../data/raw/landuse/pluto_sample.geojson', dtype={'BBL':str})

# currently not applied, filtering lots that has total commercial area more than 0
# gdf_pluto_manhattan_commercial = gdf_pluto_manhattan.loc[gdf_pluto_manhattan.loc[:,'ComArea']>0]
gdf_pluto_manhattan_commercial = gdf_pluto_manhattan.copy()
gdf_pluto_manhattan_commercial.loc[:,'BBL'] = gdf_pluto_manhattan_commercial.loc[:,'BBL'].astype('int64').astype(str)

C:\Users\seung\AppData\Local\Temp\ipykernel_21568\692124823.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1012700029' '1004470038' '1012800030' ... '1003480022' '1019177501'
 '1010160054']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf_pluto_manhattan_commercial.loc[:,'BBL'] = gdf_pluto_manhattan_commercial.loc[:,'BBL'].astype('int64').astype(str)


In [13]:
# merge the building footprint and mappluto data. Office, residential and retail area will be included in the building footprint
gdf_building_manhattan_commercial = gdf_building_manhattan.merge(pd.DataFrame(gdf_pluto_manhattan_commercial.loc[:,['BBL',
                                                                                                                    'OfficeArea',
                                                                                                                    'RetailArea',
                                                                                                                    'ResArea']]), left_on='mpluto_bbl', right_on='BBL', how='inner').drop('BBL', axis=1)

In [14]:
gdf_building_manhattan_commercial = gdf_building_manhattan_commercial.drop_duplicates(subset=['bin','globalid'])

In [15]:
gdf_building_manhattan_commercial

,name,base_bbl,shape_area_left,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,geometry,shape_area,OfficeArea,RetailArea,ResArea
0,None,1006220001,0.0,47.93,1006220001,1949,{1BA099B1-6FDA-48F8-BDFF-7D96CF6AF127},Constructed,2100,22,Photogramm,1011281,2017-08-22,139945,"MULTIPOLYGON (((-74.00571 40.73543, -74.00571 ...",636520502.758,0,3000,12594
1,None,1017540024,0.0,61.92,1017540024,1889,{A35DA67B-9202-4669-9B9A-B148A4510B11},Constructed,2100,20,Photogramm,1054234,2020-01-28,604770,"MULTIPOLYGON (((-73.93815 40.80831, -73.93824 ...",636520502.758,0,0,0
2,None,1015550003,0.0,27.02,1015550003,1920,{532C7007-8BC1-419C-85A5-E3E2444A57A6},Constructed,2100,22,Photogramm,1050239,2017-08-22,322279,"MULTIPOLYGON (((-73.94854 40.78184, -73.94852 ...",636520502.758,0,4443,0
3,None,1013470032,0.0,16.86,1013470032,1930,{96D5188A-97A7-4F20-99E7-73ADCF345B24},Constructed,2100,39,Photogramm,1081198,2009-02-14,357713,"MULTIPOLYGON (((-73.96460 40.75698, -73.96464 ...",636520502.758,0,0,0
4,None,1001070028,0.0,92.39088895,1001070028,2010,{73023EDE-4BD2-46C9-BEA6-6A2B213BF648},Constructed,2100,5,Photogramm,1088490,2017-08-22,1257098,"MULTIPOLYGON (((-74.00059 40.70837, -74.00080 ...",636520502.758,0,4100,27615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6031,None,1019390040,0.0,72.21,1019390040,1911,{7C22798D-5D1E-48B9-816E-ED2A14822013},Constructed,2100,30,Photogramm,1058818,2020-01-28,624147,"MULTIPOLYGON (((-73.94497 40.81486, -73.94499 ...",636520502.758,0,0,0
6032,None,1010330025,0.0,383.0,1010330025,2022,{BAAEA650-D38F-4F44-A6DF-F27812CC7C21},Marked for Construction,5100,44,Other (Man,1024945,2023-10-17,1296650,"MULTIPOLYGON (((-73.99045 40.75830, -73.98998 ...",636520502.758,0,20364,347570
6033,None,1012100033,0.0,145.60084588,1012107504,2016,{F9FAB5C4-22C5-43CC-A987-460EF258B5F3},Constructed,2100,84,Photogramm,1031764,2017-08-22,720808,"MULTIPOLYGON (((-73.97538 40.78238, -73.97544 ...",636520502.758,0,7516,51257
6034,None,1014970028,0.0,93.81,1014970028,1925,{9510EFEC-1C81-4C81-8AC9-15AB98F59E80},Constructed,2100,95,Photogramm,1046832,2017-08-22,839008,"MULTIPOLYGON (((-73.95827 40.78010, -73.95833 ...",636520502.758,4500,0,98150


In [16]:
gdf_building_manhattan_commercial = gdf_building_manhattan_commercial.drop(['name', 'base_bbl', 'shape_area_left','lststatype', 'feat_code', 'groundelev','lstmoddate', 'doitt_id','geomsource'], axis=1)

In [18]:
gdf_building_manhattan_commercial.to_file('../../data/processed/building/building_sample.geojson', driver='GeoJSON')